# Load Required Packages

In [ ]:
import pandas as pd
import numpy as np
import gc
import pickle

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import HDBSCAN
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Load

In [ ]:
Folder = "E:\\datasets\\car\\"

In [ ]:
Sample = pd.read_parquet(f"{Folder}SampleEncoded.parq")
Sample

In [ ]:
X = pd.read_parquet(f"{Folder}X.parq")
X = X.fillna(0)

y = pd.read_parquet(f"{Folder}y.parq")

In [ ]:
X_old = pd.read_parquet(f"{Folder}X_old.parq")
X_old = X_old.fillna(0)

y_old = pd.read_parquet(f"{Folder}y_old.parq")

# Dimensionality Reduction

In [ ]:
pca = PCA(n_components=2).fit(X)

# Scan for outliers

In [ ]:
dbs = HDBSCAN(min_cluster_size=5, leaf_size=100)

In [ ]:
dbs = dbs.fit(EncodedData.iloc[0:10000])

In [ ]:
dbs.labels_

# Split into Training & Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.40, random_state=1991
)

if "X" in globals():
    del globals()["X"]

if "y" in globals():
    del globals()["y"]

gc.collect()

# K Neighbors Regressor

In [ ]:
kn = KNeighborsRegressor(25, weights="distance")
kn.fit(X_train, y_train)

In [ ]:
with open("PriceScaler.obj", "rb") as PS_File:
    PriceScaler = pickle.load(PS_File)

In [ ]:
Start = 150000
Number = 500
y_pred = kn.predict(X_test.iloc[Start : Start + Number])

In [ ]:
Comparison = pd.concat(
    [pd.DataFrame(y_pred), y_test.iloc[Start : Start + Number].reset_index(drop=True)],
    axis=1,
)
Comparison = pd.DataFrame(
    PriceScaler.inverse_transform(Comparison), columns=["Predicted", "Price"]
)
print(len(Comparison))
display(Comparison.head())
r2_score(Comparison["Price"], Comparison["Predicted"])

# DBScan

In [ ]:
db = DBSCAN(min_samples=100)
db.fit(X)

# Decision Tree Regressor

In [ ]:
reg = DecisionTreeRegressor().fit(X, y)
y_pred = reg.predict(X)
r2_score(y_pred, y)

In [ ]:
# dummies["Y"] = y
# dummies["Pred"] = y_pred
# dummies["Diff"] = (dummies["Y"] - dummies["Pred"]).abs()
# gc.collect()
# Data = pd.read_parquet("clean_df.parq")
# Data["Pred"] = dummies["Pred"]
# Data["Diff"] = dummies["Diff"]
# Data = Data.sort_values(by="Diff", ascending=False)
# Data.to_parquet("PredictionDifferences.parq")
# Data[(Data["maker"] == "volkswagen") & (Data["model"] == "tiguan")]["price_eur"].hist()